In [1]:
%reload_ext autoreload
%autoreload 2

from config import Config
from transcriptLoader import getCombinedTranscripts 

config = Config()
config.setFromEnv()

videoNames = [
    "New Quizzes Video",
    "Rearrange Playlist video",
    "IMSE 514 Presentation",
    "New Google Assignments in Canvas",
    "Piazza Introduction Workshop",
]

videoToUse = "New Google Assignments in Canvas"
transcriptToUse = getCombinedTranscripts(
    videoToUse,
    captionsFolder="Captions",
)

In [2]:
from topicExtractor import retrieveTopics

topicsOverTime, topicModel = retrieveTopics(videoToUse, transcriptToUse, config=config)

c:\Users\takposha\AppData\Local\anaconda3\envs\annoto-gai\Lib\site-packages\bertopic\vectorizers\_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)
